# 크롤링(Crawling)
- 웹페이지에서 필요한 부분 추출

In [15]:
url = "https://www.kma.go.kr/wid/queryDFSRSS.jsp?zone=1121571000"

import requests

res = requests.get(url)

# res.text

from bs4 import BeautifulSoup
bs = BeautifulSoup(res.text,"html.parser") # XML 분해

지역명 = bs.select_one("category").string # 한 개 태그 검색용 -> 글자만 뽑고 싶으면 .string 사용
# bs.select("") # 여러개 태그 검색용 : 리스트 반환

print(지역명)

서울특별시 광진구 화양동


In [22]:
# 일기예보데이터는 <data>에 3시간단위로 저장
data_list = bs.select('data')
print('갯수:', len(data_list))
data_arr = []
day_dict = {'0' : '오늘', '1' : '내일', '2' : '모레'}

for dt in data_list:
    data_dict = dict() #일기정보 저장용
    
    data_dict['hour'] = dt.select_one('hour').string
    data_dict['day'] = day_dict[dt.select_one('day').string]
    data_dict['wfkor']= dt.select_one('wfkor').string
    data_arr.append(data_dict)

data_arr

갯수: 19


[{'hour': '18', 'day': '오늘', 'wfkor': '흐림'},
 {'hour': '21', 'day': '오늘', 'wfkor': '구름 많음'},
 {'hour': '24', 'day': '오늘', 'wfkor': '구름 많음'},
 {'hour': '3', 'day': '내일', 'wfkor': '구름 많음'},
 {'hour': '6', 'day': '내일', 'wfkor': '맑음'},
 {'hour': '9', 'day': '내일', 'wfkor': '맑음'},
 {'hour': '12', 'day': '내일', 'wfkor': '맑음'},
 {'hour': '15', 'day': '내일', 'wfkor': '맑음'},
 {'hour': '18', 'day': '내일', 'wfkor': '맑음'},
 {'hour': '21', 'day': '내일', 'wfkor': '맑음'},
 {'hour': '24', 'day': '내일', 'wfkor': '맑음'},
 {'hour': '3', 'day': '모레', 'wfkor': '구름 많음'},
 {'hour': '6', 'day': '모레', 'wfkor': '구름 많음'},
 {'hour': '9', 'day': '모레', 'wfkor': '구름 많음'},
 {'hour': '12', 'day': '모레', 'wfkor': '구름 많음'},
 {'hour': '15', 'day': '모레', 'wfkor': '구름 많음'},
 {'hour': '18', 'day': '모레', 'wfkor': '구름 많음'},
 {'hour': '21', 'day': '모레', 'wfkor': '구름 많음'},
 {'hour': '24', 'day': '모레', 'wfkor': '흐림'}]

### KOVO 경기결과 크롤링

In [26]:
url = 'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=018&yymm=2022-08&s_part=0&r_round='
import requests # 따로 실행하기 위해서 import 한번 더 해줌

res = requests.get(url, verify=False).text


C:\Users\Admin\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.kovo.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [51]:
from bs4 import BeautifulSoup
bs = BeautifulSoup(res, 'html.parser')
tbl = bs.select_one('table.lst_schedule')# 공백이 들어가면 자식 태그가 선택됨 .은 클래스를 의미
tbl_tr = tbl.select('tr') # 경기결과가 있는 행 여러개

for tr in tbl_tr:
    if len(tr.select('td.tright')) == 0:
        continue # td.tright 가 없으면 넘어감
    game_date = tr.select_one('td.date').get_text()
    game_result = game_date + ' ' + tr.select_one('td.tright').get_text() + tr.select_one('td.tleft').get_text()
    print(game_result)

10. 16 (토) 대한항공  3 : 1  우리카드
 GS칼텍스  3 : 0  흥국생명
10. 17 (일) 현대캐피탈  3 : 1  OK금융그룹
 현대건설  3 : 1  IBK기업은행
10. 19 (화) 삼성화재  0 : 3  한국전력
 페퍼저축은행  1 : 3  KGC인삼공사
10. 20 (수) KB손해보험  3 : 2  현대캐피탈
 한국도로공사  0 : 3  현대건설
10. 21 (목) OK금융그룹  3 : 2  우리카드
 IBK기업은행  1 : 3  흥국생명
10. 22 (금) 삼성화재  3 : 0  대한항공
 페퍼저축은행  0 : 3  GS칼텍스
10. 23 (토) 한국전력  3 : 1  KB손해보험
 KGC인삼공사  3 : 0  한국도로공사
10. 24 (일) 우리카드  1 : 3  현대캐피탈
 현대건설  3 : 1  흥국생명
10. 26 (화) OK금융그룹  3 : 1  KB손해보험
 IBK기업은행  1 : 3  한국도로공사
10. 27 (수) 현대캐피탈  3 : 2  대한항공
 GS칼텍스  1 : 3  현대건설
10. 28 (목) 우리카드  3 : 0  한국전력
 KGC인삼공사  3 : 1  흥국생명
10. 29 (금) KB손해보험  2 : 3  삼성화재
 한국도로공사  3 : 0  페퍼저축은행
10. 30 (토) 대한항공  3 : 0  OK금융그룹
 GS칼텍스  3 : 1  IBK기업은행
10. 31 (일) 한국전력  3 : 0  현대캐피탈
 KGC인삼공사  0 : 3  현대건설
